#QnA Bot from Youtube Playlist

This project enables users to generate transcripts for videos in a YouTube playlist, create embeddings, and ask questions about the video content.


## Features

- Extracts video links from a YouTube playlist.
- Generates transcripts for each video.
- Creates embeddings for the transcripts.
- Provides a Q&A bot to answer questions about the video content.

#### To run this code follow these steps one by one:

### Step 1:
Install the necessary dependencies:

In [ ]:
!pip install chromadb
!pip install youtube_transcript_api
!pip install llama-index-llms-gemini
!pip install llama-index-vector-stores-chroma
!pip install llama-index-embeddings-huggingface
!pip install llama-index-readers-youtube-transcript
!pip install llama-index-core
!pip install google-api-python-client
!pip install speechrecognition
!pip install gradio
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.7 MB/s eta 0:00:00


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

### Step 2:

Define a function that accepts a YouTube playlist URL and generates a list of individual YouTube video links from that playlist.

In [ ]:
import googleapiclient.discovery
from urllib.parse import parse_qs, urlparse

def get_playlist_items(url):
        # Extract the playlist ID from the provided URL
        query = parse_qs(urlparse(url).query, keep_blank_values=True)
        playlist_id = query["list"][0]

        # Build the YouTube API client using the API key
        youtube = googleapiclient.discovery.build("youtube", "v3", developerKey="your_youtube_api_key")

        # Create an API request to get playlist items
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,
            maxResults=50  # Maximum number of results to return per request
        )

        playlist_items = []
        while request is not None:
            # Execute the request and get the response
            response = request.execute()
            # Add the items from the response to the playlist_items list
            playlist_items += response["items"]
            # Get the next page of results, if available
            request = youtube.playlistItems().list_next(request, response)

        # Extract the video links from the playlist items
        links = [
            f'https://www.youtube.com/watch?v={t["snippet"]["resourceId"]["videoId"]}&list={self.playlist_id}'
            for t in playlist_items
        ]
        return links

### Step 3:

Define a function that accepts a YouTube video link and returns the transcript of the video.

In [ ]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

def generate_Transcript(url):
    loader = YoutubeTranscriptReader()

    documents = loader.load_data(
      ytlinks= [url]
    )
    return documents

### Step 4:

Define a funciton that accepts a transcript, generates embeddings from it, and stores these embeddings in a ChromaDB database.

In [ ]:
import chromadb
from llama_index.core import (
    Settings, StorageContext, VectorStoreIndex
)
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

def generate_embeddings(data):

        # Set embedding and language models
        embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
        llm = Gemini(api_key="your_gemini_api_key", model_name="models/gemini-pro")

        # Load documents
        documents = data

        # Create a client and a new collection
        client = chromadb.PersistentClient(path='./chroma_db')
        chroma_collection = client.get_or_create_collection("quickstart")

        # Create a vector store
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

        # Create a storage context
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        # Set Global settings
        Settings.llm = llm
        Settings.embed_model = embedding_model

    # Create an index from the documents and save it to the disk
        index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context
    )

### Step 5:

Define a function to generate answers by querying the index with a given question.

In [ ]:
from llama_index.core import Settings, VectorStoreIndex, StorageContext
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

def generate_answers(question):

        # Initialize the Gemini embedding model
        embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

        # Initialize the Gemini language model
        llm = Gemini(api_key="your_gemini_api_key", model_name="models/gemini-pro")

        # Set Global settings
        Settings.llm = llm
        Settings.embed_model = embedding_model

        # Load the ChromaDB client
        client = chromadb.PersistentClient(path='./chroma_db')

        # Fetch the collection from ChromaDB
        chroma_collection = client.get_collection("quickstart")

        # Fetch the vector store from the collection
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

        # Create a storage context
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        # Get the index from the vector store
        index = VectorStoreIndex.from_vector_store(vector_store)

        query_engine = index.as_query_engine()
        return query_engine.query(question)

### Step 6:

Run each of the functions in the specified order to ensure proper initialization and functionality.

In [ ]:
import gradio as gr
import whisper
import time

whisper_model = whisper.load_model("base")

# Function to process YouTube playlist and generate transcripts
def process_playlist(url):
    links = get_playlist_items(url)

    transcripts = []
    num_videos = len(links)

    # Create a progress tracker instance
    progress_tracker = gr.Progress(num_videos)

    for i, link in enumerate(links, start=1):
        transcript = generate_Transcript(link)
        transcripts.append(f"Video {i}:\n {transcript}")

        generate_embeddings(transcript)

        # Update progress
        progress_tracker.update(i)
        time.sleep(1)

    return "Processing Completed."

# Function to generate answers from text input
def generate_text_answer(question):
    if question.lower() == "exit":
        return "Session terminated."

    answer = generate_answers(question)
    return answer

# Function to generate answers from voice input
def generate_voice_answer(audio):
    # Load the audio file
    audio_file = whisper.load_audio(audio)
    result = whisper_model.transcribe(audio_file, verbose=True)
    question = result["text"]

    if question.lower() == "exit":
        return "Session terminated."

    answer = generate_answers(question.lower())
    return answer

# Gradio interface
def gradio_interface():
    with gr.Blocks(theme=gr.themes.Default(primary_hue=gr.themes.colors.red, secondary_hue=gr.themes.colors.pink)) as demo:
        with gr.Row():
            gr.Image(value="./utils/logo.jpeg", label="YouTube QnA Bot", type="filepath", scale=1)
            gr.Label("YouTube QnA Bot", scale=4)

        with gr.Row():
            url_input = gr.Textbox(label="Enter YouTube playlist URL", placeholder="YouTube playlist URL")
            process_button = gr.Button("Process Playlist", variant="primary")

        # Add a progress bar and status output
        progress_tracker = gr.Progress()
        status_output = gr.Textbox(label="Status", placeholder="Processing status will appear here", lines=2)

        # Define the function that updates the progress
        def update_progress(url):
            status = process_playlist(url)
            return status

        process_button.click(fn=update_progress, inputs=url_input, outputs=status_output)

        with gr.Row():
            answer_output = gr.Textbox(label="Answer", placeholder="Answer will appear here", lines=5)

        with gr.Row():
            audio_input = gr.Audio(sources="microphone", type="filepath", scale=4)
            ask_voice_button = gr.Button("Ask (Voice)", scale=1, variant="primary")

        with gr.Row():
            question_text_input = gr.Textbox(label="Enter your question", placeholder="Ask your question here", scale=4)
            ask_text_button = gr.Button("Ask (Text)", scale=1, variant="primary")

        ask_text_button.click(generate_text_answer, inputs=question_text_input, outputs=answer_output)
        ask_voice_button.click(generate_voice_answer, inputs=audio_input, outputs=answer_output)

    return demo

# Launch the Gradio interface
gradio_interface().launch(share=True)